#GLM Case Study

In this tutorial, we will present a detailed case study in the context of auto liability insurance, showcasing a variety of GLM techniques.

Let's start by loading the libraries that are going to be helpful. We're going to rely on the statistical learning toolkit ski-cit learn, which provides GLM functionalty but also will be used in the context of other ML models. It is less comfortable to use than some of the other packages. But it is versatile and fast, and therefore one of the most popular predictive modeling toolkits.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.linear_model import GammaRegressor

We consider predictive modeling of auto claims, i.e. the overarching challenge is predicting frequencies and/or severities of claims.  We rely on the comprehensive French Motor Third-Part Liability datasets `ferMTPLfreq` and `ferMTPLsev` available within the [package CASdatasets](http://cas.uqam.ca/).

Let's take a peak, first at the frequency dataset:

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
dat_frq1 = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_III_7_freMTPLfreq1.csv')
dat_frq2 = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_III_7_freMTPLfreq2.csv')
dat_frq = pd.concat([dat_frq1,dat_frq2])
dat_frq.head()

In [ ]:
dat_frq.describe()

In [ ]:
pd.crosstab(index=dat_frq['ClaimNb'], columns="count")

In [ ]:
pd.crosstab(index=dat_frq['ClaimNb'], columns="count").plot(kind='bar')

So, as expected, multiple claims are rare. The vast majority of cases don't have a claim.

Let's look at the severities:

In [ ]:
dat_sev = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_III_7_freMTPLsev.csv')
print(dat_sev.shape)
dat_sev.head()

In [ ]:
dat_sev.describe()

So, again, as expected, we have a few very large claims.

## Merge the Data Sets

Since there are multiple claims for each policy, let's summarize the claims to the policy level, so as to allow for an easy merge:

In [ ]:
df = dat_sev.groupby('PolicyID', as_index=False).agg({"ClaimAmount":"mean"})
df

Now we can simply merge the frequency and the severity sets into our master data set, where we set the `NA` `ClaimAmount` entries to zero where we don't have claims:

In [ ]:
dat = pd.merge(dat_frq, dat_sev.groupby('PolicyID', as_index=False).agg({"ClaimAmount":"mean"}),how='left')
dat = dat.fillna(0)
dat.head()

## Building a GLM

As previously, we need to put the categorical variables to dummies. For that, we separate the dummies and the numerical variables, make the dummies, and then concatenate.

In [ ]:
dummies = pd.get_dummies(dat[dat.columns[[4,7,8,9]]],drop_first=True)
dat = dat.drop(dat.columns[[0, 1, 4, 7, 8, 9]], axis=1)
dat = pd.concat([dat,dummies], axis=1)
dat.head()

Let's do some visualizations of some of the variables:

In [ ]:
plt.hist(dat['Exposure'])
plt.show()

In [ ]:
plt.hist(dat['DriverAge'])
plt.show()

...likely what we expected...

In [ ]:
plt.hist(dat['CarAge'])
plt.show()

...also no surprises...

Let's look at population density, that's a more interesting variable:

In [ ]:
plt.hist(dat['Density'])
plt.show()

So it looks like very small and a few very high densities. Let's go to log-scale.

In [ ]:
dat.loc[:, 'Density'] = np.log(dat['Density'])
plt.hist(dat['Density'])
plt.show()

Let's also check out our targets:

In [ ]:
dat.loc[dat['ClaimAmount']>0,'ClaimAmount'].quantile([.9, .95, .99, .999])

It is possible that the few very large claims will cause trouble, so let's cut off at 50K:

In [ ]:
dat['ClaimAmount'][dat['ClaimAmount']>50000] = 50000

Let's define two target variables: the number of claims and the loss amount

In [ ]:
y_freq = dat['ClaimNb']
y_sev = dat['ClaimAmount']
X = dat.drop(columns = ['ClaimNb','ClaimAmount'])

And let's model frequencies via a Poisson Regression:

In [ ]:
freqmodel = PoissonRegressor()
freqmodel.fit(X,y_freq)
preds_freq = freqmodel.predict(X)
np.corrcoef(preds_freq,y_freq)

The correlation is fairly low, but when we look at a scatter plot...

In [ ]:
plt.scatter(y_freq,preds_freq)

it does seem like that the claims with higher frequencies have higher predictions, though they are still close to zero. Accurately predicting car accidents is just very difficult.

Let's looks at claims:

In [ ]:
plt.hist(dat['ClaimAmount'])
plt.show()

and non-zero claims on a log-scale.

In [ ]:
plt.hist(np.log(dat.loc[dat['ClaimAmount']>0,'ClaimAmount']))
plt.show()

Let's run a Gamma regression for the severities:

In [ ]:
X_sev = dat.loc[dat['ClaimAmount']>0,:]
y_sev = X_sev['ClaimAmount']
X_sev = X_sev.drop(columns = ['ClaimNb','ClaimAmount'])
sevmodel = GammaRegressor()
sevmodel.fit(X_sev,y_sev)
plt.scatter(y_sev,sevmodel.predict(X_sev))

So we're kind-of catching the trend.

Now we can fuse together by muliplying predicted frequencies and severities:

In [ ]:
preds_sev = sevmodel.predict(X)
preds_tot = preds_freq * preds_sev

In [ ]:
plt.scatter(y_freq * dat['ClaimAmount'],preds_tot)